# **Fine Tunning With RoBert**

In [1]:
import json
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForMultipleChoice, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

# Load training data
with open('/kaggle/input/haik-24-question-answering/train.json', 'r') as f:
    train_data = json.load(f)

def pad_options(options, target_length=5, placeholder="No answer"):
    return options + [placeholder] * (target_length - len(options))

# Apply padding to all items in train_data
for item in train_data:
    item['options'] = pad_options(item['options'])

# Verify the result
for item in train_data:
    assert len(item['options']) == 5, f"Question {item['question']} does not have 5 options"

print("All questions now have exactly 5 options.") 

with open('/kaggle/input/haik-24-question-answering/test.json', 'r') as f:
    test_data = json.load(f)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def preprocess_data(data):
    processed_data = []
    for item in data:
        question_key = next(key for key in item.keys() if key.startswith('question '))
        processed_item = {
            'question': item[question_key],
            'options': item['options'],
            'label': item.get('label', None)  # None for test data
        }
        processed_data.append(processed_item)
    return processed_data

train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)

class QADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        question = item['question']
        options = item['options']
        
        inputs = self.tokenizer(
            [question] * len(options),
            options,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        
        if item['label'] is not None:
            inputs['labels'] = torch.tensor(item['label'] - 1)  # Subtract 1 as labels are 1-indexed
        
        return inputs

# Split train data into train and validation
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

train_dataset = QADataset(train_data, tokenizer)
val_dataset = QADataset(val_data, tokenizer)

# Training
# Initialize model
model = AutoModelForMultipleChoice.from_pretrained("roberta-base")

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return {"accuracy": (predictions == eval_pred.label_ids).astype(np.float32).mean().item()}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Submission

# Apply padding to all items in train_data
for item in test_data:
    item['options'] = pad_options(item['options'])

# Verify the result
for item in test_data:
    assert len(item['options']) == 5, f"Question {item['question']} does not have 5 options"

print("All questions now have exactly 5 options.") 
test_dataset = QADataset(test_data, tokenizer)

# Predict on test set
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1) + 1  # Add 1 to make it 1-indexed

# Create submission file
submission = pd.DataFrame({
    'question': [f'question {i}' for i in range(len(test_data))],
    'label': predicted_labels
})
submission.to_csv('submission.csv', index=False)

2024-06-28 11:55:17.933652: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-28 11:55:17.933756: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-28 11:55:18.068150: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


All questions now have exactly 5 options.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]